#Imports and Uploads

In [ ]:
'''
The four files need to be uploaded onto the colab environment:
COG.csv, PHENO.csv, MRI.csv, DEMO.csv
It can be uploaded by running this cell
'''
from google.colab import files
m,p,d,i = files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from keras.models import Model
from keras.layers import Input
from keras.layers import GRU
from scipy.stats import pearsonr
from sklearn.impute import KNNImputer

In [ ]:
#Once the files are uploaded onto the environment
pheno = pd.read_csv('PHENO.csv')
mri = pd.read_csv('MRI.csv')
demo = pd.read_csv('DEMO.csv')
cog = pd.read_csv('COG.csv')

In [ ]:
pheno = pheno.sort_values(by = ['RID'])
print("Pheno: \n")
print(pheno.head())
mri = mri.sort_values(by = ['RID'])
print("\n MRI: \n")
print(mri.head())
demo = demo.sort_values(by = ['RID'])
print("\n Demo: \n")
print(demo.head())
cog = cog.sort_values(by = ['RID'])
print("\n Cog: \n")
print(cog.head())

##Demo Formatting

In [ ]:
'''
Formatting the data as well as removing the data which contain too many missing 
values.
'''
Age = []
Apoe4 = []
labels_demo = []
start = 2
rids_demo = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 Age
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 Apoe4
cnt = 0
for index,row in demo.iterrows():
  if start != row['RID']:    
    if sum(l1) != 0 and sum(l2) != 0 and cnt>1:
      rids_demo.append(row['RID'])
      Age.append(l1)
      Apoe4.append(l2)
      if dec == 'AD':
        labels_demo.append(1)
      else:
        labels_demo.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  ptm = row['AGE']
  ptt = row['APOE4']
  if m == 'bl':
    cnt+=1
    l1[0] = ptm
    l2[0] = ptt
  elif m == 'm06':
    cnt+=1
    l1[1] = ptm
    l2[1] = ptt
  elif m == 'm12':
    cnt+=1
    l1[2] = ptm
    l2[2] = ptt
  elif m == 'm24':
    cnt+=1
    l1[3] = ptm
    l2[3] = ptt
  elif m == 'm36':
    cnt+=1
    l1[4] = ptm
    l2[4] = ptt
  elif m == 'm48':
    cnt+=1
    l1[5] = ptm
    l2[5] = ptt

In [ ]:
x1 = np.array(Age)
x2 = np.array(Apoe4)
#Length of data in Cog
print(len(x1),len(x2))
y_demo = np.array(labels_demo)

In [ ]:
#KNN Imputation is done to fill in the few missing values which are left
imputer = KNNImputer(n_neighbors=28, missing_values=0, weights = 'distance')
x1_demo = imputer.fit_transform(x1)  #Age
x2_demo = imputer.fit_transform(x2)  #APOE4

In [ ]:
lAge = x1_demo.tolist()
lApoe4 = x2_demo.tolist()
print(lAge)
print(lApoe4)

##Cog Formatting

In [ ]:
'''
Formatting the data as well as removing the data which contain too many missing 
values.
'''
CogPtMem = []
CogPtTotal = []
labels_cog = []
start = 2
rids_cog = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTMEM
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTTOT
cnt = 0
for index,row in cog.iterrows():
  if start != row['RID']:    
    if sum(l1) != 0 and sum(l2) != 0 and cnt>1:
      rids_cog.append(row['RID'])
      CogPtMem.append(l1)
      CogPtTotal.append(l2)
      if dec == 'AD':
        labels_cog.append(1)
      else:
        labels_cog.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  ptm = row['EcogPtMem']
  ptt = row['EcogPtTotal']
  if m == 'bl':
    cnt+=1
    l1[0] = ptm
    l2[0] = ptt
  elif m == 'm06':
    cnt+=1
    l1[1] = ptm
    l2[1] = ptt
  elif m == 'm12':
    cnt+=1
    l1[2] = ptm
    l2[2] = ptt
  elif m == 'm24':
    cnt+=1
    l1[3] = ptm
    l2[3] = ptt
  elif m == 'm36':
    cnt+=1
    l1[4] = ptm
    l2[4] = ptt
  elif m == 'm48':
    cnt+=1
    l1[5] = ptm
    l2[5] = ptt

In [ ]:
x1 = np.array(CogPtMem)
x2 = np.array(CogPtTotal)
#Length of data in Cog
print(len(x1),len(x2))
y_cog = np.array(labels_cog)

In [ ]:
#KNN Imputation is done to fill in the few missing values which are left
imputer = KNNImputer(n_neighbors=28, missing_values=0, weights = 'distance')
x1_cog = imputer.fit_transform(x1)  #CogPtMem
x2_cog = imputer.fit_transform(x2)  #CogPtTotal

In [ ]:
lCogPtMem = x1_cog.tolist()
lCogPtTotal = x2_cog.tolist()
print(lCogPtMem)
print(lCogPtTotal)

##Pheno Formatting

In [ ]:
'''
Formatting the data as well as removing the data which contain too many missing 
values.
'''
abeta = []
tau = []
ptau = []
labels_pheno = []
start = 3
rids_pheno = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 abeta
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 tau
l3 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 ptau
cnt = 0
for index,row in pheno.iterrows():
  if start != row['RID']:    
    if sum(l1) != 0 and sum(l2) != 0 and sum(l3) != 0 and cnt>1:
      rids_pheno.append(row['RID'])
      abeta.append(l1)
      tau.append(l2)
      ptau.append(l3)
      if dec == 'AD':
        labels_pheno.append(1)
      else:
        labels_pheno.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    l3 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  abt = row['ABETA']
  ta = row['TAU']
  pta = row['PTAU']
  if m == 'bl':
    cnt+=1
    l1[0] = abt
    l2[0] = ta
    l3[0] = pta
  elif m == 'm06':
    cnt+=1
    l1[1] = abt
    l2[1] = ta
    l3[1] = pta
  elif m == 'm12':
    cnt+=1
    l1[2] = abt
    l2[2] = ta
    l3[2] = pta
  elif m == 'm24':
    cnt+=1
    l1[3] = abt
    l2[3] = ta
    l3[3] = pta
  elif m == 'm36':
    cnt+=1
    l1[4] = abt
    l2[4] = ta
    l3[4] = pta
  elif m == 'm48':
    cnt+=1
    l1[5] = abt
    l2[5] = ta
    l3[5] = pta

In [ ]:
x1_pheno = np.array(abeta)
x2_pheno = np.array(tau)
x3_pheno = np.array(ptau)
y_pheno = np.array(labels_pheno)
print(len(x1_pheno))

In [ ]:
#Imputation
imputer = KNNImputer(n_neighbors=16, missing_values=0, weights = 'distance')
x1_pheno = imputer.fit_transform(x1_pheno) #ABETA
x2_pheno = imputer.fit_transform(x2_pheno) #TAU
x3_pheno = imputer.fit_transform(x3_pheno) #PTAU

In [ ]:
labeta = x1_pheno.tolist()
ltau = x2_pheno.tolist()
lptau = x3_pheno.tolist()
print(labeta)
print(ltau)
print(lptau)

##MRI Formatting

In [ ]:
hippo = []
entor = []
labels_mri = []
start = 2
rids_mri = []
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTMEM
l2 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTTOT
cnt = 0
for index,row in mri.iterrows():
  if start != row['RID']:   
    if sum(l1) != 0 and sum(l2) != 0 and cnt>1:
      rids_mri.append(row['RID'])
      hippo.append(l1)
      entor.append(l2)
      if dec == 'AD':
        labels_mri.append(1)
      else:
        labels_mri.append(0)
    l1 = [0,0,0,0,0,0]
    l2 = [0,0,0,0,0,0]
    start = row['RID']
    cnt=0
  dec = row['DX_bl']
  m = row['VISCODE']
  hi = row['Hippocampus']
  en = row['Entorhinal']
  if m == 'bl':
    cnt+=1
    l1[0] = hi
    l2[0] = en
  elif m == 'm06':
    cnt+=1
    l1[1] = hi
    l2[1] = en
  elif m == 'm12':
    cnt+=1
    l1[2] = hi
    l2[2] = en
  elif m == 'm24':
    cnt+=1
    l1[3] = hi
    l2[3] = en
  elif m == 'm36':
    cnt+=1
    l1[4] = hi
    l2[4] = en
  elif m == 'm48':
    cnt+=1
    l1[5] = hi
    l2[5] = en

In [ ]:
x1_mri = np.array(hippo)
x2_mri = np.array(entor)
y_mri = np.array(labels_mri)
print(len(x1_mri),len(x2_mri))

In [ ]:
imputer = KNNImputer(n_neighbors=32, missing_values=0, weights = 'distance')
x1_mri = imputer.fit_transform(x1_mri) #Hippocampus
x2_mri = imputer.fit_transform(x2_mri) #Entorhinal

In [ ]:
lhippo = x1_mri.tolist()
lentor = x2_mri.tolist()
print(lhippo)
print(lentor)

##Adding Patients of different modalities to each other

In [ ]:
#Adding patients from Pheno rids to Cog rids
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTMEM and PTTOT
cnt = -1
for i in rids_pheno:
  cnt = cnt + 1
  if i not in rids_cog:
    rids_cog.append(i)
    lCogPtMem.append(l1)
    lCogPtTotal.append(l1)
    labels_cog.append(labels_pheno[cnt])

#Adding patients from MRI rids to Cog rids
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 PTMEM and PTTOT
cnt = -1
for i in rids_mri:
  cnt = cnt + 1
  if i not in rids_cog:
    rids_cog.append(i)
    lCogPtMem.append(l1)
    lCogPtTotal.append(l1)
    labels_cog.append(labels_mri[cnt])

#Adding Patients from Cog rids to Pheno rids
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 abeta, tau, ptau
cnt = -1
for i in rids_cog:
  cnt = cnt + 1
  if i not in rids_pheno:
    rids_pheno.append(i)
    labeta.append(l1)
    ltau.append(l1)
    lptau.append(l1)
    labels_pheno.append(labels_cog[cnt])

#Adding Patients from MRI rids to Pheno rids
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 abeta, tau, ptau
cnt = -1
for i in rids_mri:
  cnt = cnt + 1
  if i not in rids_pheno:
    rids_pheno.append(i)
    labeta.append(l1)
    ltau.append(l1)
    lptau.append(l1)
    labels_pheno.append(labels_mri[cnt])

#Adding patients from Cog rids to MRI rids
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 hippo, entor
cnt = -1
for i in rids_cog:
  cnt = cnt + 1
  if i not in rids_mri:
    rids_mri.append(i)
    lhippo.append(l1)
    lentor.append(l1)
    labels_mri.append(labels_cog[cnt])

#Adding patients from Pheno rids to MRI rids
l1 = [0,0,0,0,0,0] #bl, m06, m12, m24, m36, m48 hippo, entor
cnt = -1
for i in rids_pheno:
  cnt = cnt + 1
  if i not in rids_mri:
    rids_mri.append(i)
    lhippo.append(l1)
    lentor.append(l1)
    labels_mri.append(labels_pheno[cnt])

print(len(rids_cog))
print(len(rids_pheno))
print(len(rids_mri))
print(len(labels_cog))
print(len(labels_pheno))
print(len(labels_mri))

##Handle Demo Addition

In [ ]:
# using dictionary comprehension 
# to convert lists to dictionary 
d_Age = {rids_demo[i]: lAge[i] for i in range(len(rids_demo))} 
d_Apoe4 = {rids_demo[i]: lApoe4[i] for i in range(len(rids_demo))} 
d_Labels = {rids_demo[i]: labels_demo[i] for i in range(len(rids_demo))} 

In [ ]:
print(len(d_Age))
print(len(d_Apoe4))
print(len(d_Labels))
print(d_Age)
print(d_Apoe4)
print(d_Labels)

In [ ]:
rids_demos = []
lAges = []
lApoes4 = []
l1 = [0,0,0,0,0,0]

print(len(rids_demo))
print(len(lAge))
print(len(lApoe4))
for i in rids_cog:
  if i in d_Age.keys():
    rids_demos.append(i)
    lAges.append(d_Age[i])
    lApoes4.append(d_Apoe4[i])
  elif i not in d_Age.keys():
    rids_demos.append(i)
    lAges.append(l1)
    lApoes4.append(l1)

print(len(rids_demos))
print(len(lAges))
print(len(lApoes4))

In [ ]:
x1 = np.array(lAges)
x2 = np.array(lApoes4)
#Length of data in Demo
print(len(x1),len(x2))

In [ ]:
#KNN Imputation is done to fill in the few missing values which are left
imputer = KNNImputer(n_neighbors=28, missing_values=0, weights = 'distance')
x1_demo = imputer.fit_transform(x1)  #Age
x2_demo = imputer.fit_transform(x2)  #Apoe4

##GRU for Demo Final

In [ ]:
#GRUs require 3 dimensional input
x1_demo = x1_demo.reshape((1,1776,6))
x2_demo = x2_demo.reshape((1,1776,6))

In [ ]:
#Generating feature vectors
inputs0 = Input(shape=(1776, 6))
gru0 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru0(inputs0)
model0 = Model(inputs=inputs0, outputs=whole_sequence_output)

In [ ]:
lage_fv = model0.predict(x1_demo) #Age: fv = feature vector
lapoe4_fv = model0.predict(x2_demo) #APOE4

##Converting Demo Feature Vectors and Labels to Dictionaries

In [ ]:
lage_fv_list = lage_fv[0].tolist()
print(lage_fv_list)
lapoe4_fv_list = lapoe4_fv[0].tolist()
print(lapoe4_fv_list)

dic_lage = {rids_demos[i]: lage_fv_list[i] for i in range(len(rids_demos))}
dic_lapoe4 = {rids_demos[i]: lapoe4_fv_list[i] for i in range(len(rids_demos))}

##Imputing COG values for additional patients

In [ ]:
x1 = np.array(lCogPtMem)
x2 = np.array(lCogPtTotal)
#Length of data in Cog
print(len(x1),len(x2))

In [ ]:
#KNN Imputation is done to fill in the few missing values which are left
imputer = KNNImputer(n_neighbors=28, missing_values=0, weights = 'distance')
x1_cog = imputer.fit_transform(x1)  #CogPtMem
x2_cog = imputer.fit_transform(x2)  #CogPtTotal

###GRU for COG final

In [ ]:
#GRUs require 3 dimensional input
x1_cog = x1_cog.reshape((1,1776,6))
x2_cog = x2_cog.reshape((1,1776,6))

In [ ]:
#Generating feature vectors
inputs1 = Input(shape=(1776, 6))
gru1 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru1(inputs1)
model1 = Model(inputs=inputs1, outputs=whole_sequence_output)

In [ ]:
cogptmem_fv = model1.predict(x1_cog) #CogPtMem: fv = feature vector
cogpttotal_fv = model1.predict(x2_cog) #CogPtTotal
print(cogptmem_fv)
print(cogpttotal_fv)

##Converting Cog Feature Vectors and Labels to Dictionaries

In [ ]:
cogptmem_fv_list = cogptmem_fv[0].tolist()
print(cogptmem_fv_list)
cogpttotal_fv_list = cogpttotal_fv[0].tolist()
print(cogpttotal_fv_list)

dic_cogptmem = {rids_cog[i]: cogptmem_fv_list[i] for i in range(len(rids_cog))}
dic_cogpttotal = {rids_cog[i]: cogpttotal_fv_list[i] for i in range(len(rids_cog))}
dic_cog_labels = {rids_cog[i]: labels_cog[i] for i in range(len(rids_cog))}

In [ ]:
print(dic_cogptmem)
print(dic_cogpttotal)
print(dic_cog_labels)

##Imputing Pheno values for additional patients

In [ ]:
x1_pheno1 = np.array(labeta)
x2_pheno1 = np.array(ltau)
x3_pheno1 = np.array(lptau)
print(len(x1_pheno1),len(x2_pheno1),len(x3_pheno1))

In [ ]:
#Imputation
imputer = KNNImputer(n_neighbors=16, missing_values=0, weights = 'distance')
x1_pheno = imputer.fit_transform(x1_pheno1) #ABETA
x2_pheno = imputer.fit_transform(x2_pheno1) #TAU
x3_pheno = imputer.fit_transform(x3_pheno1) #PTAU

In [ ]:
print(x1_pheno)
print(x2_pheno)
print(x3_pheno)

###GRU for Pheno final

In [ ]:
#GRUs require 3 dimensional input
x1_pheno = x1_pheno.reshape((1,1776,6))
x2_pheno = x2_pheno.reshape((1,1776,6))
x3_pheno = x3_pheno.reshape((1,1776,6))

In [ ]:
#Normalizing all data
x1_pheno = x1_pheno/400.0
x2_pheno = x2_pheno/200.0
x3_pheno = x3_pheno/50.0

In [ ]:
#Generating feature vectors
inputs2 = Input(shape=(1776, 6))
gru2 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru2(inputs2)
model2 = Model(inputs=inputs2, outputs=whole_sequence_output)

In [ ]:
abeta_fv = model2.predict(x1_pheno)
tau_fv = model2.predict(x2_pheno)
ptau_fv = model2.predict(x3_pheno)
print(abeta_fv)
print(tau_fv)
print(ptau_fv)

##Converting Pheno Feature Vectors and Labels to Dictionaries

In [ ]:
abeta_fv_list = abeta_fv[0].tolist()
print(abeta_fv_list)
tau_fv_list = tau_fv[0].tolist()
print(tau_fv_list)
ptau_fv_list = ptau_fv[0].tolist()
print(ptau_fv_list)

dic_abeta = {rids_pheno[i]: abeta_fv_list[i] for i in range(len(rids_pheno))}
dic_tau = {rids_pheno[i]: tau_fv_list[i] for i in range(len(rids_pheno))}
dic_ptau = {rids_pheno[i]: ptau_fv_list[i] for i in range(len(rids_pheno))}
dic_pheno_labels = {rids_pheno[i]: labels_pheno[i] for i in range(len(rids_pheno))}

In [ ]:
print(dic_abeta)
print(dic_tau)
print(dic_ptau)
print(dic_pheno_labels)

##Imputing MRI values for additional patients

In [ ]:
x1_mri1 = np.array(lhippo)
x2_mri1 = np.array(lentor)
print(len(x1_mri1),len(x2_mri1))

In [ ]:
imputer = KNNImputer(n_neighbors=32, missing_values=0, weights = 'distance')
x1_mri = imputer.fit_transform(x1_mri1) #Hippocampus
x2_mri = imputer.fit_transform(x2_mri1) #Entorhinal

In [ ]:
print(x1_mri)
print(len(x1_mri))
print(x2_mri)
print(len(x2_mri))

###GRU for MRI final

In [ ]:
#Reshaping for GRU
x1_mri = x1_mri.reshape((1,1776,6))
x2_mri = x2_mri.reshape((1,1776,6))

In [ ]:
#Normalizing all data
x1_mri = x1_mri/2000
x2_mri = x2_mri/2000

In [ ]:
#Generating feature vectors
inputs3 = Input(shape=(1776, 6))
gru3 = GRU(6, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru3(inputs3)
model3 = Model(inputs=inputs3, outputs=whole_sequence_output)

In [ ]:
hippo_fv = model3.predict(x1_mri)
entor_fv = model3.predict(x2_mri)
print(hippo_fv)
print(entor_fv)

##Converting MRI Feature Vectors and Labels to Dictionaries

In [ ]:
hippo_fv_list = hippo_fv[0].tolist()
print(hippo_fv_list)
entor_fv_list = entor_fv[0].tolist()
print(entor_fv_list)

dic_hippo = {rids_mri[i]: hippo_fv_list[i] for i in range(len(rids_mri))}
dic_entor = {rids_mri[i]: entor_fv_list[i] for i in range(len(rids_mri))}
dic_mri_labels = {rids_mri[i]: labels_mri[i] for i in range(len(rids_mri))}

In [ ]:
print(dic_hippo)
print(dic_entor)
print(dic_mri_labels)

##Join All Modalitites FVs

In [ ]:
final_list = []
final_labels_list = []
for i in dic_cogptmem.keys():
  mergedlist = []
  mergedlist = dic_abeta[i] + dic_tau[i] + dic_ptau[i] + dic_cogptmem[i] + dic_cogpttotal[i] + dic_hippo[i] + dic_entor[i] #+ dic_lapoe4[i] + dic_lage[i]
  final_list.append(mergedlist)
  final_labels_list.append(dic_cog_labels[i])

print(len(final_list))
print(len(final_list[0]))
print(len(final_labels_list))

In [ ]:
positive = 0
negative = 0
for i in final_labels_list:
  if i == 1:
    positive = positive + 1
  else:
    negative = negative + 1

print("Positive: "+ str(positive))
print("Negative: "+ str(negative))

In [ ]:
total_final_list = []
total_final_labels = []
counter = 0
for i in range(len(final_labels_list)):
  if final_labels_list[i] == 1:
    total_final_list.append(final_list[i])
    total_final_labels.append(final_labels_list[i])
  elif final_labels_list[i] == 0:
    if counter < 294:
      total_final_list.append(final_list[i])
      total_final_labels.append(final_labels_list[i])
      counter = counter + 1
print(len(total_final_list))
print(total_final_labels)

##Central GRU Layer

In [ ]:
total_final_list1 = np.array(total_final_list)
#Reshaping for 3D input
tots = total_final_list1.reshape((1,588,42))

In [ ]:
#Generating feature vectors
inputs3 = Input(shape=(588, 42))
gru3 = GRU(42, return_sequences=True, return_state=True)
whole_sequence_output, final_state = gru3(inputs3)
model3 = Model(inputs=inputs3, outputs=whole_sequence_output)

In [ ]:
total_final_list_fv = model3.predict(tots)

In [ ]:
print(len(total_final_list_fv[0]))

In [ ]:
from sklearn.model_selection import train_test_split
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(total_final_list_fv[0], np.array(total_final_labels), test_size=0.4,random_state=109)

#SVM Model

In [ ]:
#SVM
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import svm
rf_params = {
    'C': [1,10, 100],
    "kernel":['linear','poly','rbf','sigmoid']
}
clf = svm.SVC(gamma='scale')
grid = GridSearchCV(clf, rf_params, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))

In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
#Create a svm Classifier
clf = svm.SVC(C=100,kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
!pip install scikit-plot

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, permutation_test_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred)
disp = plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(clf, X_test, y_test)
plt.show()

In [ ]:
#P-value
_, _, p_value = permutation_test_score(clf, X_test, y_test, scoring="accuracy", cv=None, n_permutations=200)

print(p_value)

#Random Forest Model

In [ ]:
#Random Forest
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50],
    #'min_samples_leaf': [1,2,4,8],
    #"bootstrap":[True,False],
    "criterion":['gini','entropy']
}
model = RandomForestClassifier(random_state=0)
grid = GridSearchCV(model, rf_params, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(criterion='entropy',
                               max_depth=15,
                               n_estimators=200, 
                               bootstrap = True,
                               max_features = 0.5)
# Fit on training data
model.fit(X_train, y_train)

In [ ]:
# Actual class predictions
y_pred = model.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred)
disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(model, X_test, y_test)
plt.show()

In [ ]:
#P-value
_, _, p_value = permutation_test_score(model, X_test, y_test, scoring="accuracy", cv=None, n_permutations=200)

print(p_value)

#Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data
import pandas as pd
import numpy as np

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

#create a dictionary of all values we want to test
param_grid = { 'criterion':['gini','entropy'],'max_depth': np.arange(3, 10)}
# decision tree model
dtree_model=DecisionTreeClassifier()
#use gridsearch to test all values
dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=5)
#fit model to data
dtree_gscv.fit(X_train, y_train)
print(dtree_gscv.best_params_)
print("Accuracy:"+ str(dtree_gscv.best_score_))

In [ ]:
dt = DecisionTreeClassifier(criterion='gini',max_depth=5)
dt.fit(X_train, y_train)

In [ ]:
y_pred = dt.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

#Printing Confusion Matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", conf_matrix )

#Printing F1 Score
f1score = f1_score(y_test, y_pred, average='binary')
print("F1 score: ", f1score)

#Printing Cohen-Kappa score
ckscore = cohen_kappa_score(y_test, y_pred)
print("Cohen-Kappa Score: ", ckscore)

#Printing Precision-Recall Curve
average_precision = average_precision_score(y_test, y_pred)
disp = plot_precision_recall_curve(dt, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


#Printing ROC Curve
metrics.plot_roc_curve(dt, X_test, y_test)
plt.show()


#P-value
_, _, p_value = permutation_test_score(dt, X_test, y_test, scoring="accuracy", cv=None, n_permutations=200)

print(p_value)

##Making Combined Graphs

In [ ]:
#ROC Curve

from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()


from sklearn.metrics import plot_roc_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_test, y_test, ax=ax)

In [ ]:
#Precision Recall Curve

from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure()


from sklearn.metrics import plot_precision_recall_curve
classifiers = [clf, dt, model]
ax = plt.gca()
for i in classifiers:
    plot_precision_recall_curve(i, X_test, y_test, ax=ax)